In [231]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#  **Importing the needed libraries**

In [232]:
#for data visualizing
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import style
%matplotlib inline

# **Reading the csv file**

In [233]:
#READING THE CSV FILES OF OUR DATASET
roadAccident=pd.read_csv("../input/roadaccident/road-accidents.csv",skiprows=9,sep="|",index_col="state")
milesDriven=pd.read_csv('../input/miledriven/miles-driven.csv',sep='|',index_col="state")
np.random.seed(0)

# **Read in and get an overview of the data**

In [234]:
#GETTING SOME INFORMATION AND AN OVERVIEW ABOUT OUR DATASET
print(roadAccident.shape)  #this return the number of rows and columns in our dataset
roadAccident.info() #to get an overview about our columns
roadAccident.head()

In [235]:
print(milesDriven.shape)
milesDriven.info()
milesDriven.head()

# **Create a textual and a graphical summary of the data**

In [236]:
#GETTING SOME INFORMATION ABOUT NUMERIC COLUMNS
milesDriven.describe()

In [237]:
#SHOWING SOME GRAPHS FOR THE GRAPHICAL SUMMARY
million_miles_annually=milesDriven["million_miles_annually"]
plt.figure(figsize=(10,10))
sns.histplot(million_miles_annually)
plt.show()

In [238]:
roadAccident.describe()

In [239]:
drvr_fatl_col_bmiles=roadAccident["drvr_fatl_col_bmiles"]
perc_fatl_speed=roadAccident["perc_fatl_speed"]
perc_fatl_alcohol=roadAccident["perc_fatl_alcohol"]
perc_fatl_1st_time=roadAccident["perc_fatl_1st_time"]
fig,ax=plt.subplots(2,2,figsize=(20,20))
sns.histplot(drvr_fatl_col_bmiles,ax=ax[0][0])
sns.histplot(perc_fatl_speed,ax=ax[0][1])
sns.histplot(perc_fatl_alcohol,ax=ax[1][0])
sns.histplot(perc_fatl_1st_time,ax=ax[1][1])
plt.show()

In [240]:
#the scatter plot matrix
print(pd.plotting.scatter_matrix(roadAccident,figsize=(20,20),diagonal='hist'))
plt.show()

# Quantify the association of features and accidents

In [241]:
#correlation matrix for the dataset of road accidents
roadAccident.corr(method="pearson")

**from the correlation matrix there is a weak correlation between all the features and target and we cas also see that the biggest correlatoin coefficient if between the two features perc_fatl_alcohol and perc_fatl_speed**

In [242]:
#the correlation between the target and features sorted in an desceding way
roadAccident.corr(method='pearson')["drvr_fatl_col_bmiles"][1:].sort_values(ascending=False)

**As we can see from the correlation matrix all the coeffficient of pearson matrix is less than 0.2**
**which means that there is a weak association between features (accident causes) and number of accidents**

# **Fit a multivariate linear regression**

In [243]:
#let's import the Linear Regression model from sklear
from sklearn.linear_model import LinearRegression
linearRegression=LinearRegression()
X=roadAccident.copy()
Y=X.pop(roadAccident.columns[0])
#the Y variable is the first column of our dataset (the target) 
#and X is contain the three other columns after dropping the first one (the features)
linearRegression.fit(X,Y)
print(X.columns)
print(linearRegression.coef_)
print(linearRegression.intercept_)

**When comparing the regression coefficients with the correlation coefficients, we see that they are slightly different**
    We can also see that for the first two coefficients they have the same sign as the corrlation coefficients but the third coefficients have the oposite sign of the correlation coefficient so we can say that we have a masking relationship between drvr_fatl_col_bmiles   and    perc_fatl_1st_time
    
    So after building this multivaraite regression we found that the relation that would be built between the variables is
    Y= -0.04180041 X1 + 0.19086404 X2 + 0.02473301 X3 + 9.064980483403303

# **Perform PCA on standardized data**

In [244]:
#let's do some Principale Compenent Analysis
X.head()

In [245]:
from sklearn.decomposition import PCA

"""
Standardize Our data by
Substracting the mean from each value
and dividing each column by it's standard deviation
"""
#the std() method returns the standard deviation of each column 
X=(X-X.mean())/X.std()
#fit the model with X and apply the dimentionality reduction on X
Pca=PCA(random_state=0).fit_transform(X)
X=pd.DataFrame(Pca,columns=X.columns,index=roadAccident.index)
X.head()

# **Visualize the first two principal components**

In [246]:
#let's visualize the first two principal components
#the first one in the column X.iloc[:,0]
#the second one int the column X.iloc[:,1]
plt.figure(figsize=(7,7))
plt.scatter(x=X.iloc[:,0],y=X.iloc[:,1])
plt.show()

# **Find clusters of similar states in the data**

In [247]:
from sklearn.cluster import KMeans
List=[]
for i in range(1,10):
    kMeans=KMeans(n_clusters=i,random_state=0)
    kMeans.fit(X)
    List+=[kMeans.inertia_]
plt.plot([i for i in range(1,10)],List,'ro-')
plt.xlabel('n_clusters')
plt.show()

**- as we can see from the scree plot above we're not able to find an optimal value for n_clusters as there's no clear elbow in the graph.**

# **KMeans to visualize clusters in the PCA scatter plot**

In [248]:
# assigning the states to three clusters
kMeans=KMeans(n_clusters=3,random_state=0)
X['cluster']=kMeans.fit_predict(X)
X['cluster']=X['cluster'].astype('category')
print(X.head())

In [249]:
sns.scatterplot(x=X.iloc[:,0],y=X.iloc[:,1],hue='cluster',data=X)
plt.show()

# **Visualize the feature differences between the clusters**

In [250]:
Xcluster=X['cluster']
X=roadAccident.copy()
X.pop(roadAccident.columns[0])
X['cluster']=Xcluster
sns.scatterplot(x=X.iloc[:,0],y=X.iloc[:,1],hue='cluster',data=X)
plt.show()

# **Compute the number of accidents within each cluster**

In [251]:
#let's calculate the number of accident 
milesDriven['total_accidents']=milesDriven['million_miles_annually']*roadAccident['drvr_fatl_col_bmiles']/1000
#joining another column from another dataset
X=X.join(milesDriven)
#extracting the neccessary columns from our data
X=X.loc[:,['cluster','total_accidents']]
sns.violinplot(data=X,x='cluster',y='total_accidents')
plt.ylabel('total fatal traffic accidents')
plt.ylim(0,6000)
plt.show()

# **Make a decision when there is no clear right choice**

    From the table above, the tree clusters 0, 1 and 2 have nearly the same total number of fatal accidents, but if we had to choose one of those clusters to be the first to concentrate on, It's better to choose the cluster 1 which has the highest total number of fatal accidents.